In [1]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky1D,GRU_NCL,LambdaLayer,biasLayer
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss
from torch.utils.data import DataLoader
from torch.optim import Adam

In [12]:
## change ##
batch_size = 16
clip = .25
block = ResidualBlockGLU
normalFun = normalize_log2
#normalFun = normalize_log
name = 'Spec_2d_cnn_shareWeight'

In [4]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values

Prepare data

In [5]:
train_gen = SequenceGenSpec(train,normalFun=normalFun)
#train_gen = mixupWrapper(SequenceGenSpec(train),4e-2) # does not improve
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGenSpec(train,False,normalFun=normalFun)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [6]:
model = Sequential(Dropout(0.5),
                      block(129),
                      Dropout(0.5),
                      block(258),
                      Dropout(0.5),
                      block(516),
                      Dropout(0.5),
                      block(1032),
                      Dropout(0.5),
                      LambdaLayer(lambda x:x.transpose(1,2)),
                      Linear(2064,1,bias=False),
                      LeakyReLU(0.2,True),
                      LambdaLayer(lambda x:x.squeeze(2).mean(1,keepdim=True)),
                      #biasLayer((1,)),
                      Linear(1,1),
                      LambdaLayer(lambda x:x.squeeze(1))
                      ).to('cuda:0')

In [7]:
opt = Adam(trainable_parameter(model),lr=4e-4)

In [8]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [9]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:2.52684853799068, val_loss:2.481733798980713
epoch:1, train_loss:2.10625273333146, val_loss:2.3760292530059814
epoch:2, train_loss:2.075445686395352, val_loss:2.4136786460876465
epoch:3, train_loss:2.0415587866535554, val_loss:2.4039034843444824
epoch:4, train_loss:2.0209936207303634, val_loss:2.4062561988830566
Training completed in 48.16830563545227s


Save Model/Validation

In [10]:
save_ModelandValidation(model,val_gen,name)

Submission

In [17]:
make_submission(name,model,normalFun,batch_size)